# King Benchmark Case 1

## Isoviscous thermal convection with TALA formulation.

Two-dimensional, compressible, bottom heated, steady isoviscous thermal convection in a 1 x 1 box, see case 1 of [King et al. (2009)](https://doi.org/10.1111/j.1365-246X.2009.04413.x) / [Blankenbach et al. (1989) benchmark](https://academic.oup.com/gji/article/98/1/23/622167).

Keywords: Truncated Anelastic Liquid Approximation, TALA, Convection


In [1]:
import petsc4py
from petsc4py import PETSc

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import os 
import numpy as np
import sympy
from sympy.vector import gradient, divergence, dot

### Set parameters to use 

In [2]:
Di = 0.5 # dissipation factor - FIXME: in terms of the physical parameters or do some checks on the physical meanings
Ra = 1e4 #### Rayleigh number

# other non-dimensional parameters
gamma_r = 1.    # reference Gruneisen parameter TODO: set-up to correct values
alpha_bar = 1   # this is set to 1 in the paper 
c_p = 1         # set to 1 for now
T_s = 0.091     # non-dimensionalized surface temp (TODO: 273 K/3000 K as per Davies et al)
rho_0  = 1      # non-dimensionalized surface density (TODO: set-up to correct value)

k = 1.0 #### diffusivity

boxLength = 1.0
boxHeight = 1.0
tempMin   = 0.
tempMax   = 1.
deltaTemp = tempMax - tempMin

viscosity = 1

res= 16             ### x and y res of box
nsteps = 100        ### maximum number of time steps to run the first model 
epsilon_lr = 1e-8   ### criteria for early stopping; relative change of the Vrms in between iterations  


### Create mesh and variables

In [3]:
meshbox = uw.meshing.UnstructuredSimplexBox(
                                                minCoords=(0.0, 0.0), 
                                                maxCoords=(boxLength, boxHeight), 
                                                cellSize=1.0 /res, 
                                                regular=False, 
                                                qdegree = 3
                                        )

#meshbox = uw.meshing.StructuredQuadBox(minCoords=(0.0, 0.0), maxCoords=(boxLength, boxHeight),  elementRes=(res,res))

# the reference curves are prepared here
# we follow the reference and set z = 0 to be the upper surface and z = 1 to be the lower surface
# this means that z = 0 is set at total temperature of 0 while z = 1 is set to total temp = 1
# we just need to reverse this during plotting


Processing gmsh file .meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.0625_regFalse.msh
Mesh saved to .meshes/uw_simplexbox_minC(0.0, 0.0)_maxC(1.0, 1.0)_csize0.0625_regFalse.msh.h5


## Reference values 
The temperature, T, pressure, p, and density are expressed as a sum of a reference state and a departure from this state:

\begin{aligned}
T = \bar T + T'\space \; \; \; p = \bar p + p'  \; \; \;  \rho = \bar \rho (\bar T, \bar p) + \rho'    
\end{aligned}

Where the overbarred quantities are the reference states, and the primed quantities are the perturbations. The reference states are temporally static and varies with depth, z.  

In [4]:
# the non-dimensionalized reference states are defined here
# formulations from King et al. and Davies et al.  
x, z = meshbox.X
rho_bar     = rho_0*sympy.exp(Di*z/gamma_r)         # rho_0 is non-dimensionalized density at surface TODO: double check this
temp_bar    = T_s*(sympy.exp(Di*z) - 1)            # T_s is non-dimensionalized surface temperature  

In [5]:
# visualise the mesh if in a notebook / serial

if uw.mpi.size == 1:
    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 750]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    pv.global_theme.camera["viewup"] = [0.0, 1.0, 0.0]
    pv.global_theme.camera["position"] = [0.0, 0.0, -5.0]
    pv.global_theme.show_edges = True
    pv.global_theme.axes.show = True

    meshbox.vtk("tmp_box_mesh.vtk")
    pvmesh = pv.read("tmp_box_mesh.vtk")
    pl = pv.Plotter()

    # pl.add_mesh(pvmesh,'Black', 'wireframe', opacity=0.5)
    pl.add_mesh(pvmesh, edge_color="Black", show_edges=True)

    pl.show(cpos="xy")

/Users/jcgraciosa/mambaforge/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2500: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/jcgraciosa/mambaforge/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2468: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/91/xdyy27357hv1tymzym7xhf5h0000gq/T/ipykernel_40701/1359165157.py:11: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'dbf92729-ded2-4d2c-a374-d768b6ea93df': {'defs': …

In [25]:
v_soln = uw.discretisation.MeshVariable("U", meshbox, meshbox.dim, degree=2)
p_soln = uw.discretisation.MeshVariable("P", meshbox, 1, degree=1)
t_soln = uw.discretisation.MeshVariable("T", meshbox, 1, degree=3)
t_0 = uw.discretisation.MeshVariable("T0", meshbox, 1, degree=3)

# additional variable for the gradient
dTdZ = uw.discretisation.MeshVariable(r"\partial T/ \partial \Z", # FIXME: Z should not be a function of x, y, z 
                                      meshbox, 
                                      1, 
                                      degree = 3) 

# variable containing stress in the z direction
sigma_zz = uw.discretisation.MeshVariable(r"\sigma_{zz}",  
                                        meshbox, 
                                        1, degree=2)


# ### some projection objects to calculate the
# x, y = meshbox.X

# t_soln_grad = uw.systems.Projection(meshbox, dTdZ)
# delT = t_soln.sym.diff(y)[0]
# t_soln_grad.uw_function = delT
# t_soln_grad.smoothing = 1.0e-3
# t_soln_grad.petsc_options.delValue("ksp_monitor")

Variable with name U already exists on the mesh - Skipping.
Variable with name P already exists on the mesh - Skipping.
Variable with name T already exists on the mesh - Skipping.
Variable with name T0 already exists on the mesh - Skipping.
Variable with name \partial T/ \partial \Z already exists on the mesh - Skipping.
Variable with name \sigma_{zz} already exists on the mesh - Skipping.


### System set-up (Stokes)
In the Truncated Anelastic Liquid Approximation, the conservation of mass is expressed as: 
\begin{aligned}
\nabla \cdot (\bar \rho \vec u) = \nabla \cdot \vec u + \frac{1}{\bar \rho} \vec u \nabla \bar \rho = 0.
\end{aligned}
While the conservation of momentum is given as: 
\begin{aligned}
\nabla \cdot [\eta (\nabla \vec u + \nabla \vec u^T - \frac {2}{3}\nabla \cdot \vec u \bold I)] - \nabla p' - Ra \bar \rho \bar g \bar \alpha (T - \bar T)= 0.
\end{aligned}
The deviatoric strain tensor, $\tau$, is: 
\begin{aligned}
\tau = 2 \eta \dot \epsilon = \eta (\nabla \vec u + \nabla \vec u^T - \frac {2}{3}\nabla \cdot \vec u \bold I) 
\end{aligned}

 

In [7]:
# Create Stokes object

stokes = Stokes(
    meshbox,
    velocityField=v_soln,
    pressureField=p_soln,
    solver_name="stokes",
)

# stokes petsc parameters
# stokes.petsc_options["ksp_monitor"] = None
# stokes.petsc_options['pc_type'] = 'lu'
# stokes.petsc_options["snes_max_it"] = 1000

# stokes.petsc_options["snes_atol"] = 1e-6
# stokes.petsc_options["snes_rtol"] = 1e-6


#stokes.petsc_options["ksp_rtol"]  = 1e-5 # reduce tolerance to increase speed

# Set solve options here (or remove default values
# stokes.petsc_options.getAll()

#stokes.petsc_options["snes_rtol"] = 1.0e-6
# stokes.petsc_options["fieldsplit_pressure_ksp_monitor"] = None
# stokes.petsc_options["fieldsplit_velocity_ksp_monitor"] = None
# stokes.petsc_options["fieldsplit_pressure_ksp_rtol"] = 1.0e-6
# stokes.petsc_options["fieldsplit_velocity_ksp_rtol"] = 1.0e-2
# stokes.petsc_options.delValue("pc_use_amat")

# additional term in constitutive equation? 
stokes.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(meshbox.dim)
stokes.constitutive_model.Parameters.viscosity=viscosity
# stokes.saddle_preconditioner = 1.0 / viscosity
# TODO: constitutive equation

# Free-slip boundary conditions
stokes.add_dirichlet_bc((0.0,), "Left", (0,))
stokes.add_dirichlet_bc((0.0,), "Right", (0,))
stokes.add_dirichlet_bc((0.0,), "Top", (1,))
stokes.add_dirichlet_bc((0.0,), "Bottom", (1,))

buoyancy_force = -Ra * rho_bar * alpha_bar * (t_soln.sym[0] - temp_bar) # directed downwards toward z = 0 (surface)
stokes.bodyforce = sympy.Matrix([0, buoyancy_force])

# add terms into the conservation of mass according to the formulation in King et al 
stokes.PF0 = sympy.Matrix([dot(v_soln.fn, gradient(rho_bar))/rho_bar])

stokes.stokes_problem_description() # FIXME: need to run this? 

In [26]:
stokes._Einv2

sqrt(U_{ 0,0}(N0.x, N0.y)**2/2 + U_{ 0,1}(N0.x, N0.y)**2/4 + U_{ 0,1}(N0.x, N0.y)*U_{ 1,0}(N0.x, N0.y)/2 + U_{ 1,0}(N0.x, N0.y)**2/4 + U_{ 1,1}(N0.x, N0.y)**2/2)

In [8]:
# check the equations for the Stokes system
display(stokes._p_f0) # LHS of c.o. mass 
display(stokes._u_f1) # LHS of c.o. momentum
display(stokes._u_f0) # RHS of c.o. momentum / buoyancy force

Matrix([[U_{ 0,0}(N0.x, N0.y) + 0.5*U_{ 1 }(N0.x, N0.y) + U_{ 1,1}(N0.x, N0.y)]])

Matrix([
[    -P(N0.x, N0.y) + 2*U_{ 0,0}(N0.x, N0.y), U_{ 0,1}(N0.x, N0.y) + U_{ 1,0}(N0.x, N0.y)],
[U_{ 0,1}(N0.x, N0.y) + U_{ 1,0}(N0.x, N0.y),     -P(N0.x, N0.y) + 2*U_{ 1,1}(N0.x, N0.y)]])

Matrix([[0, 10000.0*(T(N0.x, N0.y) - 0.091*exp(0.5*N0.y) + 0.091)*exp(0.5*N0.y)]])

### System set-up (Advection-Diffusion)
In the TALA formulation, the conservation of energy in terms of the total temperature, T, is expressed as:

\begin{aligned}
\frac{DT}{Dt} - \frac{Di \bar \alpha}{\bar c_p}\vec g \cdot \vec u (T + T_s) = \frac {1} {\bar \rho \bar c_p} \nabla \cdot (\bar k \nabla T) + \frac{Di}{Ra} \frac{1}{\bar \rho \bar c_p} \phi
\end{aligned}
 

In [13]:
# Create adv_diff object

adv_diff = uw.systems.AdvDiffusionSLCN(
    meshbox,
    u_Field=t_soln,
    V_Field=v_soln,
    solver_name="adv_diff",
)

adv_diff.constitutive_model = uw.systems.constitutive_models.DiffusionModel(meshbox.dim)
adv_diff.constitutive_model.Parameters.diffusivity = (k/(rho_bar*c_p))

# add source terms based needed for the EBA case - sympy expressions
#term1 = (Di/Ra)*2*viscosity*2*(stokes._Einv2**2)*(1/(rho_bar*c_p)) # FIXME: change to account for different constitutive equation, change Einv2
#term2 = (Di*alpha_bar/c_p)*v_soln.sym[1]*(t_soln.sym[0] + T_s) # FIXME: change the variable names
#term3 = (Di**2/(rho_bar*c_p))*t_soln.sym[0] # FIXME: term not in the Davies formulation
# put to the left hand side of the advection diffusion equation
#adv_diff.f = term1 - term2 #- term3 

adv_diff.theta = 0.5

# Dirichlet boundary conditions for temperature
# note that that Top is now the source of heat - deep in the mantle
# z = 0 is the surface 
adv_diff.add_dirichlet_bc(0.0, "Bottom")
adv_diff.add_dirichlet_bc(1.0, "Top")

adv_diff.petsc_options["pc_gamg_agg_nsmooths"] = 5

adv_diff.adv_diff_slcn_problem_description() # need to run this? 

In [24]:
adv_diff.estimate_dt()

  0 SNES Function norm 0.0445279 
  1 SNES Function norm 5.06683e-07 
Nonlinear SProj_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


/Users/jcgraciosa/Documents/codes/uw3-dev/underworld3/underworld3/systems/solvers.py:945: RuntimeWarning: divide by zero encountered in scalar divide
  dt_adv = min_dx / max_magvel_glob


0.00011288397954665979

### Set initial temperature field 

The initial temperature field is set to a sinusoidal perturbation. 

In [12]:
import math, sympy

pertStrength = 0.1

with meshbox.access(t_soln, t_0):
    t_soln.data[:] = 0.
    t_0.data[:] = 0.

# this is the correct one
with meshbox.access(t_soln):
    for index, coord in enumerate(t_soln.coords):
        # print(index, coord)
        pertCoeff = math.cos( math.pi * coord[0]/boxLength ) * math.sin( math.pi * (1 - coord[1])/boxLength )
    
        t_soln.data[index] = tempMin + deltaTemp*(boxHeight - (1- coord[1])) + pertStrength * pertCoeff
        t_soln.data[index] = max(tempMin, min(tempMax, t_soln.data[index]))

with meshbox.access(t_soln, t_0):
    t_0.data[:,0] = t_soln.data[:,0]


### Some plotting and analysis tools 

In [19]:
# check the mesh if in a notebook / serial
# allows you to visualise the mesh with initial perturbation

def plotFig(meshbox = meshbox, v_soln = v_soln, t_soln = t_soln, dTdZ = dTdZ, is_grad = False): 
    """
    is_grad - set flag to True if you want to plot the gradient of the temp field along z-direction 
    """
    if uw.mpi.size == 1:

        import numpy as np
        import pyvista as pv
        import vtk

        pv.global_theme.background = "white"
        pv.global_theme.window_size = [750, 250]
        pv.global_theme.antialiasing = True
        pv.global_theme.jupyter_backend = "panel"
        pv.global_theme.smooth_shading = True

        meshbox.vtk("tmp_box_mesh.vtk")
        pvmesh = pv.read("tmp_box_mesh.vtk")

        velocity = np.zeros((meshbox.data.shape[0], 3))
        velocity[:, 0] = uw.function.evaluate(v_soln.sym[0], meshbox.data)
        velocity[:, 1] = uw.function.evaluate(v_soln.sym[1], meshbox.data)

        pvmesh.point_data["V"] = velocity / 10

        points = np.zeros((t_soln.coords.shape[0], 3))
        points[:, 0] = t_soln.coords[:, 0]
        points[:, 1] = t_soln.coords[:, 1]

        point_cloud = pv.PolyData(points)

        with meshbox.access():
            if is_grad:
                point_cloud.point_data["Tp"] = uw.function.evaluate(dTdZ.fn, points[:, 0:2])
            else:
                point_cloud.point_data["Tp"] = uw.function.evaluate(t_soln.fn, points[:, 0:2])


        # point sources at cell centres

        cpoints = np.zeros((meshbox._centroids.shape[0] // 4 + 1, 3))
        cpoints[:, 0] = meshbox._centroids[::4, 0]
        cpoints[:, 1] = meshbox._centroids[::4, 1]

        cpoint_cloud = pv.PolyData(cpoints)

        pvstream = pvmesh.streamlines_from_source(
            cpoint_cloud,
            vectors="V",
            integrator_type=2,
            integration_direction="forward",
            compute_vorticity=False,
            max_steps=1000,
            surface_streamlines=True,
        )
        # with meshbox.access(t_soln):
        #     if is_grad:
        #         pvmesh.point_data["T"] = uw.function.evaluate(dTdZ.fn, points[:, 0:2])
        #     else:
        #         pvmesh.point_data["T"] = uw.function.evaluate(t_soln.fn, points[:, 0:2])

        pl = pv.Plotter()

        # pl.add_mesh(pvmesh,'Gray', 'wireframe')

        # pl.add_mesh(
        #     pvmesh, cmap="coolwarm", edge_color="Black",
        #     show_edges=True, scalars="T", use_transparency=False, opacity=0.5,
        # )

        pl.add_points(point_cloud, cmap="coolwarm", render_points_as_spheres=True, point_size=10, opacity=1)

        # pl.add_mesh(pvstream, opacity=0.5)
        # pl.add_arrows(arrow_loc2, arrow_length2, mag=1.0e-1)

        # pl.add_points(pdata)

        pl.show(cpos="xy")
        pvmesh.clear_data()
        pvmesh.clear_point_data()
        
plotFig()

/Users/jcgraciosa/mambaforge/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2500: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/jcgraciosa/mambaforge/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2468: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/91/xdyy27357hv1tymzym7xhf5h0000gq/T/ipykernel_24604/788831334.py:17: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'bff13ea4-5f42-4de2-a5b9-dda9ff1729d8': {'defs': …

In [15]:
def plot_T_mesh(filename):

    if uw.mpi.size == 1:

        import numpy as np
        import pyvista as pv
        import vtk

        pv.global_theme.background = "white"
        pv.global_theme.window_size = [750, 750]
        pv.global_theme.antialiasing = True
        pv.global_theme.jupyter_backend = "pythreejs"
        pv.global_theme.smooth_shading = True
        pv.global_theme.camera["viewup"] = [0.0, 1.0, 0.0]
        pv.global_theme.camera["position"] = [0.0, 0.0, 5.0]

        meshbox.vtk("tmp_box_mesh.vtk")
        pvmesh = pv.read("tmp_box_mesh.vtk")

        velocity = np.zeros((meshbox.data.shape[0], 3))
        velocity[:, 0] = uw.function.evaluate(v_soln.sym[0], meshbox.data)
        velocity[:, 1] = uw.function.evaluate(v_soln.sym[1], meshbox.data)

        pvmesh.point_data["V"] = velocity / 333
        pvmesh.point_data["T"] = uw.function.evaluate(t_soln.fn, meshbox.data)

        # point sources at cell centres

        cpoints = np.zeros((meshbox._centroids.shape[0] // 4, 3))
        cpoints[:, 0] = meshbox._centroids[::4, 0]
        cpoints[:, 1] = meshbox._centroids[::4, 1]
        cpoint_cloud = pv.PolyData(cpoints)

        pvstream = pvmesh.streamlines_from_source(
            cpoint_cloud,
            vectors="V",
            integrator_type=45,
            integration_direction="forward",
            compute_vorticity=False,
            max_steps=25,
            surface_streamlines=True,
        )

        points = np.zeros((t_soln.coords.shape[0], 3))
        points[:, 0] = t_soln.coords[:, 0]
        points[:, 1] = t_soln.coords[:, 1]

        point_cloud = pv.PolyData(points)

        with meshbox.access():
            point_cloud.point_data["T"] = t_soln.data.copy()

        pl = pv.Plotter()

        pl.add_mesh(
            pvmesh,
            cmap="coolwarm",
            edge_color="Gray",
            show_edges=True,
            scalars="T",
            use_transparency=False,
            opacity=0.5,
        )

        pl.add_points(point_cloud, cmap="coolwarm", render_points_as_spheres=False, point_size=10, opacity=0.5)

        pl.add_mesh(pvstream, opacity=0.4)

        pl.remove_scalar_bar("T")
        pl.remove_scalar_bar("V")

        pl.screenshot(filename="{}.png".format(filename), window_size=(1280, 1280), return_img=False)
        # pl.show()
        pl.close()

        pvmesh.clear_data()
        pvmesh.clear_point_data()

        pv.close_all()

#### RMS velocity
The root mean squared velocity, $v_{rms}$, is defined as: 


\begin{aligned}
v_{rms}  =  \sqrt{ \frac{ \int_V (\mathbf{v}.\mathbf{v}) dV } {\int_V dV} }
\end{aligned}

where $\bf{v}$ denotes the velocity field and $V$ is the volume of the box.

In [16]:
# underworld3 function for calculating the rms velocity 

def v_rms(mesh = meshbox, v_solution = v_soln): 
    # v_soln must be a variable of mesh
    v_rms = math.sqrt(uw.maths.Integral(mesh, v_solution.fn.dot(v_solution.fn)).evaluate())
    return v_rms

#print(f'initial v_rms = {v_rms()}')

### Main simulation loop

In [17]:
t_step = 0
time = 0.

timeVal =  np.zeros(nsteps)*np.nan
vrmsVal =  np.zeros(nsteps)*np.nan

In [18]:
#### Convection model / update in time


# NOTE: There is a strange interaction here between the solvers if the zero_guess is set to False


while t_step < nsteps:
    vrmsVal[t_step] = v_rms()
    timeVal[t_step] = time

    stokes.solve(zero_init_guess=True) # originally True
    delta_t = 0.5 * stokes.estimate_dt()
    adv_diff.solve(timestep=delta_t, zero_init_guess=False) # originally False

    # stats then loop
    tstats = t_soln.stats()

    if uw.mpi.rank == 0:
        print("Timestep {}, dt {}".format(t_step, delta_t))
        
        print(f't_rms = {t_soln.stats()[6]}, v_rms = {v_rms()}')

    if t_step > 1 and abs((vrmsVal[t_step] - vrmsVal[t_step - 1])/vrmsVal[t_step]) < epsilon_lr:
        break

    t_step += 1
    time   += delta_t


# save final mesh variables in the run 
# os.makedirs("../meshes", exist_ok = True)

# expt_name = "Ra1e4_res" + str(res)
# savefile = "{}_ts{}.h5".format(expt_name,t_step)
# meshbox.save(savefile)
# v_soln.save(savefile)
# t_soln.save(savefile)
# meshbox.generate_xdmf(savefile)

  0 SNES Function norm 261.125 
  1 SNES Function norm 0.000190816 
Nonlinear stokes_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 0.417901 
  1 SNES Function norm 1.11681e-05 
Nonlinear adv_diff_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
Timestep 0, dt 0.00015361076396409634
t_rms = 0.579484905552599, v_rms = 23.221761829165903
  0 SNES Function norm 261.16 
  1 SNES Function norm 0.00024485 
Nonlinear stokes_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 0.439057 
  1 SNES Function norm 9.60821e-06 
Nonlinear adv_diff_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
Timestep 1, dt 0.0001454003011339462
t_rms = 0.5796087204977513, v_rms = 24.56239379056287
  0 SNES Function norm 261.199 
  1 SNES Function norm 0.00221731 
Nonlinear stokes_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1
  0 SNES Function norm 0.461852 
  1 SNES Function norm 9.9855e-06 
Nonlinear a

### Post-run analysis

**Benchmark values**
The loop above outputs $v_{rms}$ as a general statistic for the system. For further comparison, the benchmark values for the RMS velocity, $v_{rms}$, Nusselt number, $Nu$, and non-dimensional gradients at the cell corners, $q_1$ and $q_2$, are shown below for different Rayleigh numbers. All benchmark values shown below were determined in Blankenbach *et al.* 1989 by extroplation of numerical results. 

| $Ra$ | $v_{rms}$ | $Nu$ | $q_1$ | $q_2$ |
| ------------- |:-------------:|:-----:|:-----:|:-----:|
| 10$^4$ | 42.865 | 4.884 | 8.059 | 0.589 |
| 10$^5$ | 193.215 | 10.535 | 19.079 | 0.723 |
| 10$^6$ | 833.990 | 21.972 | 45.964 | 0.877 |

In [ ]:
# set-up variables to use in calculating the benchmark values 

if do_rerun: # re-run with higher resolution mesh
    meshbox_use = meshbox_hr
    dTdZ_use = dTdZ_hr 
    t_soln_use = t_soln_hr
    sigma_zz_use = sigma_zz_hr
    sigma_zz_fn = stokes_hr.stress[1, 1]
else: # 
    meshbox_use = meshbox 
    dTdZ_use = dTdZ 
    t_soln_use = t_soln
    sigma_zz_use = sigma_zz

    sigma_zz_fn = stokes.stress[1, 1]

### define the Projection object for calculating the temperature gradient
# define upper and lower surface functions
x, z = meshbox_use.X

# gradient of temp field 
t_soln_grad = uw.systems.Projection(meshbox_use, dTdZ_use)
delT = t_soln_use.sym.diff(z)[0]
t_soln_grad.uw_function = delT
t_soln_grad.smoothing = 1.0e-3
t_soln_grad.petsc_options.delValue("ksp_monitor")

t_soln_grad.solve()

  0 SNES Function norm 0.109011 
  1 SNES Function norm 1.3591e-06 
Nonlinear SProj_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


### Calculate the $Nu$ value
The Nusselt number is defined as: 

\begin{aligned}
Nu  =   -h\frac{ \int_{0}^{l} \partial_{z}T(x, z = h) dx} {\int_{0}^{l} T(x, z = 0) dx} 
\end{aligned}

In [ ]:
# function for calculating the surface integral 
# from Ex_Shear_Band_Notch_Benchmark
 
def surface_integral(mesh, uw_function, mask_fn):

    calculator = uw.maths.Integral(mesh, uw_function * mask_fn)
    value = calculator.evaluate()

    calculator.fn = mask_fn
    norm = calculator.evaluate()

    integral = value / norm

    return integral

In [ ]:
up_surface_defn_fn = sympy.exp(-1e6*((z - 1)**2)) # at z = 1
lw_surface_defn_fn = sympy.exp(-1e6*((z)**2)) # at z = 0

# display(up_surface_defn_fn)
# display(lw_surface_defn_fn)

up_int = surface_integral(meshbox_use, dTdZ_use.sym[0], up_surface_defn_fn)
lw_int = surface_integral(meshbox_use, t_soln_use.sym[0], lw_surface_defn_fn)

Nu = -up_int/lw_int

if uw.mpi.rank == 0:
    print("Calculated value of Nu: {}".format(Nu))

Calculated value of Nu: 4.084491366061627


### Calculate the non-dimensional gradients at the cell corners, $q_i$
The non-dimensional temperature gradient at the cell corner, $q_i$, is defined as: 
\begin{aligned}
q  =  \frac{-h}{\Delta T} \left( \frac{\partial T}{\partial z} \right)
\end{aligned}
   
Note that these values depend on the non-dimensional temperature gradient in the vertical direction, $\frac{\partial T}{\partial z}$.
These gradients are evaluated at the following points:

$q_1$ at $x=0$, $z=h$; $q_2$ at $x=l$, $z=h$;

$q_3$ at $x=l$, $z=0$; $q_4$ at $x=0$, $z=0$.   

In [ ]:
# calculate q values which depend on the temperature gradient fields

q1 = -(boxHeight/(tempMax - tempMin))*uw.function.evaluate(dTdZ_use.sym[0], np.array([[0., 0.999999*boxHeight]]))[0]
q2 = -(boxHeight/(tempMax - tempMin))*uw.function.evaluate(dTdZ_use.sym[0], np.array([[0.999999*boxLength, 0.999999*boxHeight]]))[0]
q3 = -(boxHeight/(tempMax - tempMin))*uw.function.evaluate(dTdZ_use.sym[0], np.array([[0.999999*boxLength, 0.]]))[0]
q4 = -(boxHeight/(tempMax - tempMin))*uw.function.evaluate(dTdZ_use.sym[0], np.array([[0., 0.]]))[0]

if uw.mpi.rank == 0:
    print('Rayleigh number = {0:.1e}'.format(Ra))
    print('q1 = {0:.3f}; q2 = {1:.3f}'.format(q1, q2))
    print('q3 = {0:.3f}; q4 = {1:.3f}'.format(q3, q4))

Rayleigh number = 1.0e+04
q1 = 6.206; q2 = 0.647
q3 = 6.206; q4 = 0.647


### Calculate the stress for comparison with benchmark value

The stress field for whole box in dimensionless units (King 2009) is:
\begin{equation}
\tau_{ij} = \eta \frac{1}{2} \left[ \frac{\partial v_j}{\partial x_i} + \frac{\partial v_i}{\partial x_j}\right].
\end{equation}
For vertical normal stress it becomes:
\begin{equation}
\tau_{zz} = \eta \frac{1}{2} \left[ \frac{\partial v_z}{\partial z} + \frac{\partial v_z}{\partial z}\right] = \eta \frac{\partial v_z}{\partial z}.
\end{equation}
This is calculated below.

In [ ]:
# projection for the stress in the zz direction
x, y = meshbox_use.X

stress_calc = uw.systems.Projection(meshbox_use, sigma_zz_use)
stress_calc.uw_function = sigma_zz_fn
stress_calc.smoothing = 1.0e-3
stress_calc.petsc_options.delValue("ksp_monitor")
stress_calc.solve()

  0 SNES Function norm 88.3953 
  1 SNES Function norm 0.000800098 
Nonlinear SProj_2_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


The vertical normal stress is dimensionalised as: 

$$
    \sigma_{t} = \frac{\eta_0 \kappa}{\rho g h^2}\tau _{zz} \left( x, z=h\right)
$$

where all constants are defined below. 

Finally, we calculate the topography defined using $h = \sigma_{top} / (\rho g)$. The topography of the top boundary calculated in the left and right corners as given in Table 9 of Blankenbach et al 1989 are:

| $Ra$          |    $\xi_1$  | $\xi_2$  |  $x$ ($\xi = 0$) |
| ------------- |:-----------:|:--------:|:--------------:|
| 10$^4$  | 2254.02   | -2903.23  | 0.539372          |
| 10$^5$  | 1460.99   | -2004.20  | 0.529330          |
| 10$^6$  | 931.96   | -1283.80  | 0.506490          |

In [ ]:
# subtract the average value for the benchmark since the mean is set to zero 

mean_sigma_zz_top = -surface_integral(meshbox_use, 
                                     sigma_zz_use.sym[0], 
                                     up_surface_defn_fn)/boxLength


In [ ]:
# Set parameters in SI units

grav = 10        # m.s^-2
height = 1.e6    # m 
rho  = 4.0e3     # g.m^-3
kappa  = 1.0e-6  # m^2.s^-1

eta0=1.e23

def calculate_topography(coord): # only coord has local scope

    sigma_zz_top = -uw.function.evaluate(sigma_zz_use.sym[0], coord) - mean_sigma_zz_top
    
    # dimensionalise 
    dim_sigma_zz_top  = ((eta0 * kappa) / (height**2)) * sigma_zz_top
    topography = dim_sigma_zz_top / (rho * grav)
    
    return topography



In [ ]:
# topography at the top corners 
e1 = calculate_topography(np.array([[0, 0.999999*boxHeight]]))
e2 = calculate_topography(np.array([[0.999999*boxLength, 0.999999*boxHeight]]))

# calculate the x-coordinate with zero stress
with meshbox_use.access():
    cond = meshbox_use.data[:, 1] == meshbox_use.data[:, 1].max()
    up_surface_coords = meshbox_use.data[cond]
    up_surface_coords[:, 1] = 0.999999*up_surface_coords[:, 1]

abs_topo = abs(calculate_topography(up_surface_coords))

min_abs_topo_coord = up_surface_coords[np.where(abs_topo == abs_topo.min())[0]].flatten()

if uw.mpi.rank == 0:
    print("Topography [x=0], [x=max] = {0:.2f}, {1:.2f}".format(e1[0], e2[0]))
    print("x where topo = 0 is at {0:.6f}".format(min_abs_topo_coord[0]))

Topography [x=0], [x=max] = 2423.66, -3013.92
x where topo = 0 is at 0.562500
